# Parallel Factor Run (Modular)

Compute factors in parallel, then reuse results for analytics/correlations/rolling without recomputing factors.

In [1]:
from datetime import date
import logging
import sys
from pathlib import Path

try:
    root = Path(__file__).resolve().parents[1]
except NameError:
    root = Path.cwd().resolve().parents[0]
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

from quantlab_factor_library.run_factors import (
    compute_factors,
    run_analytics_only,
    compute_correlations_only,
    run_time_effects,
)
logging.basicConfig(level=logging.INFO)


## 1) Compute factors in parallel

In [2]:
# Adjust workers to your CPU/I/O; 4 is a safer default than 10
factors, ls_returns, ff, fwd_returns = compute_factors(parallel=True, max_workers=4)
print(f"Computed {len(factors)} factors; LS series: {len(ls_returns)}; FF available: {ff is not None}")


INFO:quantlab_factor_library.run_factors:Saved FF factors to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_ff_timeseries.parquet
/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_factor_library/factors/volatility.py:20: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices.pct_change()
/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_factor_library/factors/idiosyncratic_volatility.py:20: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices.pct_change()
/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_fact

Computed 51 factors; LS series: 51; FF available: True


## 2) Analytics

In [3]:
analytics = run_analytics_only(factors, fwd_returns, ff=ff, write_registry=True)
{k: {k2: res['summary'].get(k2) for k2 in ['ls_sharpe','ls_max_drawdown','ls_sharpe_last_yr','ls_max_drawdown_last_yr']} for k,res in analytics.items()}


INFO:quantlab_factor_library.analytics:Updated factor registry at /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_analytics_summary.parquet
INFO:quantlab_factor_library.analytics:Updated diagnostics reference registry at /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_analytics_summary.parquet and /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_analytics_summary.csv
INFO:quantlab_factor_library.analytics:Updated factor registry at /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_analytics_summary.parquet
INFO:quantlab_factor_library.analytics:Updated diagnostics reference registry at /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_analytics_summary.parquet and /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_analytics_summary.csv
INFO:quantlab_factor_library.analytics:Updated factor registry at /Users/edl/Documents/dev/quantlab_v2/data/factors/

{'momentum_12m': {'ls_sharpe': np.float64(-0.20288048382866425),
  'ls_max_drawdown': np.float64(-0.9172728115855876),
  'ls_sharpe_last_yr': np.float64(-0.5747597127065099),
  'ls_max_drawdown_last_yr': np.float64(-0.24287074540656295)},
 'volatility_60d': {'ls_sharpe': np.float64(0.8022260316858534),
  'ls_max_drawdown': np.float64(-0.62785523080127),
  'ls_sharpe_last_yr': np.float64(1.7063728933126825),
  'ls_max_drawdown_last_yr': np.float64(-0.253352090183682)},
 'residual_momentum_12m': {'ls_sharpe': np.float64(0.3549690172940553),
  'ls_max_drawdown': np.float64(-0.7462236851281956),
  'ls_sharpe_last_yr': np.float64(0.3229348467299529),
  'ls_max_drawdown_last_yr': np.float64(-0.1481054976329238)},
 'ivol_60d': {'ls_sharpe': np.float64(0.8832555128837725),
  'ls_max_drawdown': np.float64(-0.5179214053077941),
  'ls_sharpe_last_yr': np.float64(1.7132280767802675),
  'ls_max_drawdown_last_yr': np.float64(-0.20877996115686337)},
 'downside_vol_60d': {'ls_sharpe': np.float64(0.760

## 3) Correlations

In [4]:
corr_paths = compute_correlations_only(factors, ls_returns=ls_returns, ff=ff)
corr_paths

INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_correlation.parquet
INFO:quantlab_factor_library.run_factors:Saved factor vs FF correlation to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_ff_correlation.parquet


(PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_correlation.parquet'),
 PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_ff_correlation.parquet'))

## 4) Rolling time effects

In [5]:
rolling = run_time_effects(factors, fwd_returns, window=252, step=21)
rolling.head() if not rolling.empty else "No rolling stats"


INFO:quantlab_factor_library.run_factors:Saved rolling analytics to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_rolling_analytics.parquet


,factor,date,rolling_mean_ic,rolling_ic_ir
0,momentum_12m,2002-02-08,0.015233,0.111731
1,momentum_12m,2002-03-12,0.012520,0.089192
2,momentum_12m,2002-04-11,0.013509,0.098703
3,momentum_12m,2002-05-10,0.020592,0.158410
4,momentum_12m,2002-06-11,0.022781,0.176176
